In [1]:
from pandas import set_option as pd_set_option
pd_set_option('display.max_rows', 500)
pd_set_option('display.max_columns', 500)
from excel_app import getDataFromExcel
from pandas import concat as pd_concat
from pandas import merge as pd_merge
from excel_app import df_to_excel
from reports.contracts.contract import  get_xls_struct

def get_contract_report():
    xls_struct = get_xls_struct()
    df_period = getDataFromExcel("files//periods.xlsx", list(xls_struct[0]))
    df_period = df_period.rename(columns=xls_struct[0])
    df_rev = getDataFromExcel("files//revenew.xlsx", list(xls_struct[1]))
    df_rev = df_rev.rename(columns=xls_struct[1])
    df_exp = getDataFromExcel("files//expense.xlsx", list(xls_struct[2]))
    df_exp = df_exp.rename(columns=xls_struct[2])

    result = pd_concat([df_rev, df_exp], ignore_index=True, sort=False)
    result = result.loc[result['rev_treat'] == '"Договор 00000000052 от 25.01.2012 13:03:37"']
    result = pd_merge(result, df_period, on=["treat", "branch"], how="left")
    result = result.fillna(
        {"date": "01.01.2019 0:00:00", "pay_plan": 0, "pay_fact": 0, "revenue_plan": 0, "revenue_fact": 0}).fillna(
        "null")
    result = result.pivot_table(index=xls_struct[3], columns="date",
                                values=["pay_plan", "pay_fact", "revenue_plan", "revenue_fact"], fill_value=0,
                                margins=True,
                                aggfunc=sum)
    # result = result.swaplevel(0, 1, axis=1).sort_index(axis=1)
    result = result.reset_index()
    return result

get_contract_report()

NameError: name 'get_xls_struct' is not defined